In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df3 = pd.read_csv('/content/drive/MyDrive/Project_Msc/Project/pIC50_data/Leishmania_04_bioactivity_data_3class_pIC50.csv')

In [4]:
df3.drop("Unnamed: 0",axis=1)

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL164580,C[N+]1(CCOP(=O)([O-])OCCOC2CCCCC2)CCOCC1,inactive,351.380,1.31410,0.0,6.0,4.000000
1,CHEMBL166314,C[N+](C)(C)CCOP(=O)([O-])OCCCCCC12CC3CC(CC(C3)...,intermediate,387.501,3.97100,0.0,4.0,5.395774
2,CHEMBL165097,C[N+](C)(C)CCOP(=O)([O-])OCCCCC=C1CCCCC1,inactive,333.409,3.25500,0.0,4.0,4.000000
3,CHEMBL166092,C[N+](C)(C)CCOP(=O)([O-])OCCCCCCCCCCC=C1C2CC3C...,intermediate,469.647,6.08760,0.0,4.0,5.500313
4,CHEMBL349670,C[N+](C)(C)CCOP(=O)([O-])OCCOc1ccc2ccccc2c1,inactive,353.355,2.42640,0.0,5.0,4.000000
...,...,...,...,...,...,...,...,...
6486,CHEMBL4873840,N=C(N)Nc1ccc2[nH]c(-c3csc4ccccc34)nc2c1,active,307.382,3.74997,4.0,3.0,7.232102
6487,CHEMBL4876357,N=C(N)Nc1ccc2[nH]c(-c3ccc(-c4cccs4)s3)nc2c1,active,339.449,4.32527,4.0,4.0,7.423659
6488,CHEMBL4862955,N=C(N)Nc1ccc2[nH]c(-c3ccc(-c4ccccc4)s3)nc2c1,active,333.420,4.26377,4.0,3.0,7.356547
6489,CHEMBL4849996,N=C(N)Nc1ccc2[nH]c(-c3cc(-c4ccc(Cl)cc4)on3)nc2c1,active,352.785,3.84367,4.0,4.0,8.113509


In [5]:
y = df3.pIC50
y

0       4.000000
1       5.395774
2       4.000000
3       5.500313
4       4.000000
          ...   
6486    7.232102
6487    7.423659
6488    7.356547
6489    8.113509
6490    7.044793
Name: pIC50, Length: 6491, dtype: float64

In [6]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [7]:
! unzip /content/drive/MyDrive/Project_Msc/Project/fingerprints_xml.zip

Archive:  /content/drive/MyDrive/Project_Msc/Project/fingerprints_xml.zip
  inflating: AtomPairs2DFingerprintCount.xml  
  inflating: AtomPairs2DFingerprinter.xml  
  inflating: EStateFingerprinter.xml  
  inflating: ExtendedFingerprinter.xml  
  inflating: Fingerprinter.xml       
  inflating: GraphOnlyFingerprinter.xml  
  inflating: KlekotaRothFingerprintCount.xml  
  inflating: KlekotaRothFingerprinter.xml  
  inflating: MACCSFingerprinter.xml  
  inflating: PubchemFingerprinter.xml  
  inflating: SubstructureFingerprintCount.xml  
  inflating: SubstructureFingerprinter.xml  


In [8]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

In [9]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']
FP_List = ['MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [ ]:
fp = dict(zip(FP_list, xml_files))
fp

In [ ]:
!pip install padelpy

In [ ]:
from padelpy import padeldescriptor
for i in FP_List:

  fingerprint = i #user input

  fingerprint_output_file = ''.join([fingerprint,'.csv']) #PubChem.csv
  fingerprint_descriptortypes = fp[fingerprint]

  padeldescriptor(mol_dir='molecule.smi', 
                  d_file=fingerprint_output_file, #'Substructure.csv' example
                  #descriptortypes='SubstructureFingerprint.xml', 
                  descriptortypes= fingerprint_descriptortypes,
                  detectaromaticity=True,
                  standardizenitro=True,
                  standardizetautomers=True,
                  threads=6,
                  removesalt=True,
                  log=True,
                  fingerprints=True)
  descriptors = pd.read_csv(fingerprint_output_file)
  # descriptors
  # df_2class
  X = descriptors.drop('Name', axis=1)
   #maybe we will choose bioactivity with 3 class
  model_dataset_PubChem = pd.concat([X,y],axis=1)
  model_dataset_PubChem.to_csv(f"/content/drive/MyDrive/Project_Msc/Project/pIC50_data/Fingerprints_descriptors/model_dataset_"+fingerprint_output_file ,index=False)